In [68]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
import mne

In [69]:
np.set_printoptions(precision=4)

In [70]:
# Load the normal and abnormal ECG data from the CSV files
ptbdb_normal = pd.read_csv("C:\\Users\\91880\\Desktop\\AI PROJECTS\\ECG\\ptbdb_normal.csv", header=None)
ptbdb_abnormal = pd.read_csv("C:\\Users\\91880\\Desktop\\AI PROJECTS\\ECG\\ptbdb_abnormal.csv", header=None)


In [71]:
# Combine the normal and abnormal ECG data into a single dataframe
ptbdb_combined = pd.concat([ptbdb_normal, ptbdb_abnormal], ignore_index=True, axis=0)


In [72]:
# Reshuffle the whole new dataframe
ptbdb_combined_shuffled = ptbdb_combined.sample(frac=1, random_state=42)

In [73]:
# Generate Test and Train datasets
X = ptbdb_combined_shuffled.iloc[:, :-1]  # All values except the last column
y = ptbdb_combined_shuffled.iloc[:, -1]  # All values from the last column

train, test, train_target, test_target = train_test_split(X, y, test_size=0.20, random_state=42)

In [75]:
# Assuming your data is in a 2D array (channels x samples)
X = np.random.rand(10, 1000)  # Replace this with your actual data
info = mne.create_info(ch_names=['ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7', 'ch8', 'ch9', 'ch10'],
                        sfreq=250, ch_types='eeg')

raw = mne.io.RawArray(X, info=info, verbose=False)

In [76]:
# Bandpass filtering
raw.filter(l_freq=1, h_freq=40)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 825 samples (3.300 s)



Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,10 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,1.00 Hz
Lowpass,40.00 Hz


In [77]:
# Extract features from the filtered ECG data
features = np.abs(raw.get_data())


In [78]:
# Normalize the features using StandardScaler from sklearn
scaler = StandardScaler()
normalized_features = scaler.fit_transform(features)

In [82]:
# Create a new mne.io.RawArray object with the normalized ECG data
normalized_ecg_raw = mne.io.RawArray(normalized_features, info=info, verbose=False)

class Config:
    oversample = True
    undersample = False

oversampler = SMOTE()
undersampler = RandomUnderSampler()

if Config.oversample:
    train, train_target = oversampler.fit_resample(train, train_target)
elif Config.undersample:
    train, train_target = undersampler.fit_resample(train, train_target)
else:
    print("Using the original mitbih dataset")

In [83]:
# Import SVM Machine Learning model
model = SVC(cache_size=500)

model.fit(train, train_target)

SVC(cache_size=500)

In [84]:

predictions = model.predict(test)

In [85]:
# Accuracy metrics 
report = classification_report(test_target, predictions, digits=4)
print(report)

              precision    recall  f1-score   support

         0.0     0.7654    0.9280    0.8389       805
         1.0     0.9700    0.8913    0.9290      2106

    accuracy                         0.9014      2911
   macro avg     0.8677    0.9096    0.8839      2911
weighted avg     0.9134    0.9014    0.9041      2911

